<a href="https://colab.research.google.com/github/anshulnangru/LLM/blob/main/input_target_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
with open("/content/drive/MyDrive/Data for LLM/Jojo's Bizarre Adventure - Over Heaven.txt","r", encoding="utf-8") as f:
  raw_text=f.read()

In [2]:
import importlib
import tiktoken

In [29]:
#gpt4 tokenizer
#"gpt2" is gpt2's tokenizer
tokenizer=tiktoken.get_encoding("o200k_base")
print(len(tokenizer._mergeable_ranks))

199998


In [5]:
enc_text=tokenizer.encode(raw_text)
enc_text

[16,
 279,
 47684,
 264,
 47416,
 7931,
 49658,
 198,
 32420,
 656,
 658,
 937,
 75,
 21199,
 15,
 198,
 33151,
 656,
 7843,
 279,
 200,
 17,
 279,
 200,
 18,
 279,
 200,
 19,
 279,
 200,
 20,
 279,
 200,
 21,
 279,
 2653,
 9655,
 13,
 643,
 468,
 277,
 283,
 23393,
 5848,
 283,
 279,
 200,
 22,
 279,
 30736,
 279,
 3335,
 357,
 2059,
 357,
 9289,
 679,
 1062,
 6741,
 41059,
 1078,
 84932,
 326,
 141234,
 289,
 290,
 198,
 2878,
 28688,
 484,
 9655,
 13,
 643,
 468,
 277,
 283,
 23393,
 5848,
 283,
 4570,
 6010,
 780,
 306,
 25047,
 11,
 357,
 1481,
 2935,
 261,
 16928,
 198,
 1464,
 38091,
 13,
 13602,
 357,
 9879,
 484,
 316,
 5624,
 2617,
 15825,
 484,
 9655,
 13,
 23393,
 5848,
 283,
 8274,
 472,
 3101,
 198,
 4290,
 407,
 819,
 316,
 7668,
 306,
 495,
 2375,
 1602,
 1511,
 480,
 673,
 11,
 402,
 261,
 3211,
 328,
 5937,
 53598,
 198,
 427,
 12951,
 448,
 1952,
 945,
 18864,
 3211,
 11,
 448,
 198447,
 70,
 73462,
 326,
 108355,
 291,
 1330,
 558,
 7943,
 540,
 290,
 1268,
 328,
 2

In [6]:
enc_sample=enc_text[80:]

In [7]:
enc_sample

[6010,
 780,
 306,
 25047,
 11,
 357,
 1481,
 2935,
 261,
 16928,
 198,
 1464,
 38091,
 13,
 13602,
 357,
 9879,
 484,
 316,
 5624,
 2617,
 15825,
 484,
 9655,
 13,
 23393,
 5848,
 283,
 8274,
 472,
 3101,
 198,
 4290,
 407,
 819,
 316,
 7668,
 306,
 495,
 2375,
 1602,
 1511,
 480,
 673,
 11,
 402,
 261,
 3211,
 328,
 5937,
 53598,
 198,
 427,
 12951,
 448,
 1952,
 945,
 18864,
 3211,
 11,
 448,
 198447,
 70,
 73462,
 326,
 108355,
 291,
 1330,
 558,
 7943,
 540,
 290,
 1268,
 328,
 261,
 5594,
 328,
 93863,
 326,
 18304,
 11,
 290,
 5207,
 357,
 8486,
 18157,
 198,
 266,
 41592,
 495,
 2616,
 591,
 290,
 23451,
 158102,
 9121,
 673,
 11,
 328,
 4165,
 11,
 625,
 198,
 373,
 15312,
 316,
 484,
 328,
 9655,
 13,
 23393,
 5848,
 283,
 885,
 2208,
 8197,
 6266,
 326,
 452,
 266,
 918,
 2608,
 13,
 623,
 198,
 14555,
 158102,
 9121,
 885,
 17346,
 484,
 1354,
 1340,
 413,
 1236,
 26628,
 306,
 290,
 55166,
 198,
 14116,
 665,
 4124,
 316,
 290,
 44310,
 328,
 9655,
 13,
 23393,
 5848,
 283

In [19]:
context_size=4
x=enc_sample[:context_size]
y=enc_sample[1:context_size+1]

In [20]:
print(f"x: {x}")
print(f"y:    {y}")

x: [6010, 780, 306, 25047]
y:    [780, 306, 25047, 11]


In [31]:
for i in range (1,context_size+1):
  context=enc_sample[:i]
  desired=enc_sample[i]

  print(tokenizer.decode(context),"----->",tokenizer.decode([desired]))

iner -----> ated
inerated ----->  in
inerated in ----->  Egypt
inerated in Egypt -----> ,


In [36]:
from torch.utils.data import Dataset, DataLoader
import torch

class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    #tokenize the entire text
    token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

    #use a sliding window to chunk the book
    for i in range (0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i : i + max_length]
      target_chunk = token_ids[i + 1 : i + max_length + 1]  # Corrected target chunk slicing
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk)) # Corrected typo from tesor to tensor

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [40]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,
                         stride=128,shuffle=True, drop_last=True,
                         num_workers=0):
  #initialize the tokenizer
  tokenizer=tiktoken.get_encoding("o200k_base")

  #create dataset
  dataset=GPTDatasetV1(txt,tokenizer,max_length,stride)

  #create dataloader
  dataloader=DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers
  )
  return dataloader

In [43]:
import torch
print('PYTorch version:', torch.__version__)
dataloader=create_dataloader_v1(
    raw_text,batch_size=1,max_length=4, stride=2, shuffle=False
)

data_iter=iter(dataloader)
first_batch=next(data_iter)
print(first_batch)

PYTorch version: 2.8.0+cu126
[tensor([[   16,   279, 47684,   264]]), tensor([[  279, 47684,   264, 47416]])]


In [44]:
second_batch=next(data_iter)
print(second_batch)

[tensor([[47684,   264, 47416,  7931]]), tensor([[  264, 47416,  7931, 49658]])]


In [45]:
dataloader=create_dataloader_v1(
    raw_text,batch_size=8,max_length=256,stride=4,
    shuffle=False
)
data_iter=iter(dataloader)
inputs,targets=next(data_iter)
print("inputs: \n",inputs)
print("targets:\n",targets)

inputs: 
 tensor([[    16,    279,  47684,  ...,    472,   5693,   1218],
        [ 47416,   7931,  49658,  ...,    261,   2212,    198],
        [ 32420,    656,    658,  ...,    316,   9655,     13],
        ...,
        [   200,     17,    279,  ...,   2009,    328,    261],
        [    18,    279,    200,  ...,    316, 117339,   1078],
        [   279,    200,     20,  ...,    412, 104393,  53598]])
targets:
 tensor([[   279,  47684,    264,  ...,   5693,   1218, 136761],
        [  7931,  49658,    198,  ...,   2212,    198,  75818],
        [   656,    658,    937,  ...,   9655,     13,  23393],
        ...,
        [    17,    279,    200,  ...,    328,    261,   4010],
        [   279,    200,     19,  ..., 117339,   1078,   7562],
        [   200,     20,    279,  ..., 104393,  53598,     11]])
